<a href="https://colab.research.google.com/github/pkdyn/mechanical-syrus/blob/main/mech_syrus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [2]:
!wget --no-check-certificate \
https://github.com/pkdyn/mechanical-syrus/archive/refs/heads/main.zip 
!unzip -q /content/main.zip
!rm /content/main.zip

--2021-07-22 16:45:59--  https://github.com/pkdyn/mechanical-syrus/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/pkdyn/mechanical-syrus/zip/refs/heads/main [following]
--2021-07-22 16:46:00--  https://codeload.github.com/pkdyn/mechanical-syrus/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]  25.92K  --.-KB/s    in 0.06s   

2021-07-22 16:46:00 (416 KB/s) - ‘main.zip’ saved [26537]



In [3]:
tokenizer = Tokenizer()
max_sequence_len = 6
sentences=[]
alltext=[]
data = open('/content/mechanical-syrus-main/Moral-Sayings-of-Publius.txt').read()
corpus = data.lower()
alltext.append(corpus)
words = corpus.split(" ")
range_size = len(words)-max_sequence_len
for i in range(0, range_size):
	thissentence=""
	for word in range(0, max_sequence_len-1):
		word = words[i+word]
		thissentence = thissentence + word
		thissentence = thissentence + " "
	sentences.append(thissentence)
oov_tok = "<OOV>"
vocab_size=3200
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok, split=" ", char_level=False)
tokenizer.fit_on_texts(alltext)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in sentences:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [24]:
model = Sequential()
model.add(Embedding(total_words, 16, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=100)


Epoch 1/100
1421/1421 [==============================] - 25s 13ms/step - loss: 6.0703 - accuracy: 0.0593
Epoch 2/100
1421/1421 [==============================] - 19s 14ms/step - loss: 5.6109 - accuracy: 0.0805
Epoch 3/100
1421/1421 [==============================] - 19s 14ms/step - loss: 5.3550 - accuracy: 0.0990
Epoch 4/100
1421/1421 [==============================] - 19s 13ms/step - loss: 5.0958 - accuracy: 0.1205
Epoch 5/100
1421/1421 [==============================] - 19s 13ms/step - loss: 4.8342 - accuracy: 0.1497
Epoch 6/100
1421/1421 [==============================] - 19s 14ms/step - loss: 4.6026 - accuracy: 0.1714
Epoch 7/100
1421/1421 [==============================] - 19s 14ms/step - loss: 4.3989 - accuracy: 0.1894
Epoch 8/100
1421/1421 [==============================] - 19s 14ms/step - loss: 4.2063 - accuracy: 0.2104
Epoch 9/100
1421/1421 [==============================] - 19s 13ms/step - loss: 4.0370 - accuracy: 0.2247
Epoch 10/100
1421/1421 [==============================]

In [34]:
seed_text = "Ego is the enemy"
seed_text1= "Stillness is the key"
seed_text2= "The obstacle is the way"
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
#print(model.predict(token_list))  
predicted = model.predict_classes(token_list)
pred_classes=model.predict(token_list)
print(pred_classes.reshape(-1)[predicted])
print(predicted)
for word, index in tokenizer.word_index.items():
  if index == predicted:
    print(word)
    break

[0.16343364]
[6]
of


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [43]:

def gen(seed_text,next_words ):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word
  print(seed_text)
gen(seed_text, 20)
gen(seed_text1, 9)
gen(seed_text2, 14)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Ego is the enemy of the proud let a fool hold his tongue and he who can play the hypocrite can soonest injure his
Stillness is the key of the people's treasure he who can get more
The obstacle is the way for it one will agree with you sooner than it to it the defect
